This codesegment focuses on the following: given a camera pose of the inputimage (new_scene), find the current nodes & edges in the camera view based on the current scenegraph

general idea: we have the current mesh & scene graph a node is basically a part of the mesh -> do the ray rhooting as in the ray intersection and find the nodes that it belongs to

unsued currently!!!

In [35]:
import os 
import os.path as osp
import numpy as np 
import cv2
import open3d as o3d
import json
import pickle

#the other imports from the local stuff
#the other imports from the local stuff
import sys

ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r, ray_intersection


/local/home/ekoller/BT


In [40]:
#reading in the necessary data
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
frame_number = "000015"
img_width = 960
img_height = 540

curr_scan_id = "10b1795b-3938-2467-88fe-b10cad8913f8" #is ref 1d2f8510-d757-207c-8c48-3684433860e1
new_scan_id =  "1d2f8510-d757-207c-8c48-3684433860e1" #is rescan 9c27de56-6184-2cda-8196-591957b6387d

#the original meshes are given in the file  'labels.instances.annotated.v2.ply


In [39]:
#this method provides the list of the obj_id, global_obj ids and edges which are potentially modified in the current scene graph
#returns affected_curr_nodes, affected_curr_edges, affected_new_nodes, affected_new_edges ( not yet the global id)
def potential_nodes_and_edges(data_dir, scenes_dir, curr_scan_id, new_scan_id, frame_number):

    #compute the different transformation files
    ray_intersection.get_transformation_mask(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)

    #note to self: think about the access of the files -> is it necessary to actually store both masks ? since we have the projection already
    change_obj_new_path = os.path.join(data_dir, "mask", curr_scan_id, "obj_id","new_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg")
    change_global_new_path = os.path.join(data_dir, "mask", curr_scan_id, "global_id","new_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg")
    
    change_obj_curr_path = os.path.join(data_dir, "mask", curr_scan_id, "obj_id","curr_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg")
    change_global_curr_path = os.path.join(data_dir, "mask", curr_scan_id, "global_id","curr_val_mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg")

    #access the current edges and object id/ current ids for the tranlation
    path = os.path.join(data_dir, "files/orig/data/" "{}.pkl".format(curr_scan_id))
    with open(path, 'rb') as file:
        data = pickle.load(file)
        print("Keys in the data dictionary:", data.keys())
       

    obj2idx = data["object_id2idx"]
    edges = data["edges"]
    # obj_id = data["objects_id"]
    # global_id = data["global_objects_id"]
    # print("obj_id length", len(obj_id), "values", obj_id)
    # print("global_id length", len(global_id), "values", global_id)


    #load the files and convert them to numpyys
    new_obj = cv2.imread(change_obj_new_path, cv2.IMREAD_UNCHANGED)
    new_obj = np.array(new_obj)
    new_obj = new_obj.flatten()
    nonzero_new_obj = new_obj[new_obj!= 0]
    #this are the unique nodes which are affected
    affected_new_nodes = np.unique(nonzero_new_obj)
    #since edge access is in index also translate it
    affected_new_nodes_idx =  [obj2idx[obj_id] for obj_id in affected_new_nodes if obj_id in obj2idx]
    # new_global = cv2.imread(change_global_new_path, cv2.IMREAD_UNCHANGED)
    # new_global = np.array(new_global)


    curr_obj = cv2.imread(change_obj_curr_path, cv2.IMREAD_UNCHANGED)
    curr_obj = np.array(curr_obj)
    curr_obj = curr_obj.flatten()
    nonzero_curr_obj = new_obj[curr_obj!= 0]
    #this are the unique nodes which are affected
    affected_curr_nodes = np.unique(nonzero_curr_obj)
      #since edge access is in index also translateit
    affected_curr_nodes_idx =  [obj2idx[obj_id] for obj_id in affected_curr_nodes if obj_id in obj2idx]
    # curr_global = cv2.imread(change_global_curr_path, cv2.IMREAD_UNCHANGED)
    # curr_global = np.array(curr_global)



    affected_curr_edges_idx = [edge for edge in edges if edge[0] in affected_curr_nodes_idx or edge[1] in affected_curr_nodes_idx]
    affected_new_edges_idx = [edge for edge in edges if edge[0] in affected_new_nodes_idx or edge[1] in affected_new_nodes_idx]

    

    


    return affected_curr_nodes, affected_curr_edges_idx, affected_new_nodes, affected_new_edges_idx



In [41]:
potential_nodes_and_edges(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)

NameError: name 'scan3r' is not defined